In [1]:
import logging
import sys


import numpy

import cicada.additive
import cicada.communicator

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator, modulus=251, precision=4)
    generator = numpy.random.default_rng(seed=1234)

    for i in range(10):

        log.info("*" * 40, src=0)
        
        secret_share = protocol.uniform(generator=generator)
        secret = protocol.encoder.decode(protocol.reveal(secret_share))
        log.info(f"Player {communicator.rank} secret: {secret}")

        floor_share = protocol.floor(secret_share)
        floor = protocol.encoder.decode(protocol.reveal(floor_share))
        log.info(f"Player {communicator.rank} floor: {floor}")


main();

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:57829 from tcp://127.0.0.1:57829.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:57829 from tcp://127.0.0.1:57830.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:57829 from tcp://127.0.0.1:57831.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:root:****************************************
INFO:root:Player 0 secret: 3.5
INFO:root:Player 1 secret: 3.5
INFO:root:Player 2 secret: 3.5
INFO:root:Player 0 floor: 3.0
INFO:root:Player 1 floor: 3.0
INFO:root:Player 2 floor: 3.0
INFO:root:****************************************
INFO:root:Player 0 secret: -1.5
INFO:root:Player 1 secret: -1.5
INFO:root:Player 2 secret: -1.5
INFO:root:Player 0 floor: -1.0
INFO:root:Player 1 floor: -1.0
INFO:root:Player 2 floor: -1.0
I